In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import PolynomialFeatures

In [12]:
df=pd.read_excel('Ndata.xls')
df.shape

(45892, 16)

In [13]:
df.isnull().sum().sort_values(ascending=False)
df.head(10)

,first_name,last_name,potencial,activity,speciality,sector,locality,product,general_note,cumul_ech,cumul_cmd,cumul_sale,cumul_visit,visit_period,frequency,recency
0,Parapharmacie,Parapharmacie,B,P,PARAPHARMACIE,Medenine 2,MIDOUN JERBA,HALITOL SPRAY,NaN,0.0,0.0,0.0,1,0,0,NaN
1,Parapharmacie,Parapharmacie,B,P,PARAPHARMACIE,Medenine 2,MIDOUN JERBA,APHATAKALM GEL,NaN,0.0,0.0,0.0,1,0,0,NaN
2,Parapharmacie,Parapharmacie,B,P,PARAPHARMACIE,Medenine 2,MIDOUN JERBA,ORTHOKARE DENT,NaN,0.0,0.0,0.0,1,0,0,NaN
3,Parapharmacie,Parapharmacie,B,P,PARAPHARMACIE,Medenine 2,MIDOUN JERBA,APHTAKALM 250,NaN,0.0,0.0,0.0,1,0,0,NaN
4,Parapharmacie,Parapharmacie,B,P,PARAPHARMACIE,Medenine 2,MIDOUN JERBA,HALITOL DENT,NaN,0.0,0.0,0.0,1,0,0,NaN
5,Parapharmacie,Parapharmacie,B,P,PARAPHARMACIE,Medenine 2,MIDOUN JERBA,ORTHOKARE BB 250,NaN,0.0,0.0,0.0,1,0,0,NaN
6,Abbes,Lotfi,A,P,PHARMACIE,Kebili,KEBILI,KLOREX BB 250,NaN,0.0,0.0,0.0,1,0,0,NaN
7,Abbes,Lotfi,A,P,PHARMACIE,Kebili,KEBILI,SENSIKALM DENT,NaN,0.0,0.0,0.0,1,0,0,NaN
8,Abbes,Lotfi,A,P,PHARMACIE,Kebili,KEBILI,PAROSEPT BB 250,NaN,0.0,0.0,0.0,1,0,0,NaN
9,Abbes,Lotfi,A,P,PHARMACIE,Kebili,KEBILI,GINGISEPT DENT,NaN,0.0,0.0,0.0,1,0,0,NaN


In [14]:
df=df.dropna(subset=['cumul_cmd'])
df.shape

(45884, 16)

In [15]:
y_train= df['cumul_cmd']
X_train= df.drop('cumul_cmd', axis=1)

In [16]:
numerical_features=X_train.select_dtypes(include=['int64','float64']).columns
categorical_features=df.select_dtypes(include='object').columns

In [17]:
numerical_pipeline = make_pipeline(SimpleImputer(strategy='mean'), StandardScaler())
categorical_pipeline = make_pipeline(SimpleImputer(strategy='most_frequent'), OneHotEncoder())

In [18]:
preprocessor = make_column_transformer((numerical_pipeline, numerical_features),
                                   (categorical_pipeline, categorical_features))


In [19]:
model = make_pipeline(preprocessor,PolynomialFeatures(degree=2),LinearRegression())

In [20]:
model.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer()),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  Index(['cumul_ech', 'cumul_sale', 'cumul_visit', 'visit_period', 'frequency',
       'recency'],
      dtype='object')),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                            

In [21]:
model.score(X_train, y_train)

0.9981110539426127

In [25]:
df.columns

Index(['first_name', 'last_name', 'potencial', 'activity', 'speciality',
       'sector', 'locality', 'product', 'general_note', 'cumul_ech',
       'cumul_cmd', 'cumul_sale', 'cumul_visit', 'visit_period', 'frequency',
       'recency'],
      dtype='object')

In [89]:
import pickle
pickle.dump(model, open('RLmodel', 'wb'))

In [26]:
def predict_cmd(model, first_name, last_name, potencial, activity,
       speciality, sector, locality, product, general_note,
       cumul_ech, cumul_sale, cumul_visit,
       visit_period, frequency, recency ):
    y = np.array([first_name, last_name, potencial, activity,
       speciality, sector, locality, product, general_note,
       cumul_ech, cumul_sale, cumul_visit,
       visit_period, frequency, recency ]).reshape(1, 15)
    Y= pd.DataFrame(y,columns=X_train.columns)
    print(model.predict(Y))

In [ ]:
predict_cmd(model,'Parapharmacie','Parapharmacie','B','P','PARAPHARMACIE','Medenine 2','MIDOUN JERBA'
            ,'ORTHOKARE DENT','Prescripteur',1,0,1,0,0,0)